# Парсинг топ-250 фильмов IMDb

In [21]:
import requests
from bs4 import BeautifulSoup
import pandas as pd
import pyquery as pq

In [22]:
url = "https://ru.wikipedia.org/wiki/250_лучших_фильмов_по_версии_IMDb"

## Проверка успешности запроса


In [23]:
response = requests.get(url)
if response.status_code != 200:
    exit()

## Парсинг

In [24]:

soup = BeautifulSoup(response.text, "html.parser")
movies_inf = soup.find_all("tr")[1:]


## Извлечение данных из таблицы

Проходим по каждой строке таблицы и извлекаем информацию о фильмах в словарь.

In [25]:
movies_data = []

for row in movies_inf:
    cells = row.find_all("td")
    
    if len(cells) < 5:  
        continue  
    
    movie_ = cells[1].text.strip()
    movie_name = movie_.replace(' ', '_')
    
    url_for_description = f"https://ru.wikipedia.org/wiki/{movie_name}"
    
    response1 = requests.get(url_for_description)
    if response1.status_code == 200:
        soup_description = BeautifulSoup(response1.text, "html.parser")
        first_p = soup_description.find("p")  
        description_text = first_p.text.strip() if first_p else "Описание недоступно"
    else:
        description_text = "Ошибка при получении описания"
    
    movie = {
        'Место': cells[0].text.strip(),
        'Название': cells[1].text.strip(),
        'Год': cells[2].text.strip(),
        'Режиссёр': cells[3].text.strip(),
        'Рейтинг': cells[4].text.strip(),
        'Описание': description_text 
    }
    print(movie)
    movies_data.append(movie)

{'Место': '1', 'Название': 'Побег из Шоушенка', 'Год': '1994', 'Режиссёр': 'Фрэнк Дарабонт', 'Рейтинг': 'эпический фильм, историческая драма, тюремная драма, драма', 'Описание': '«Побе́г из Шоуше́нка» (англ.\xa0The Shawshank Redemption\xa0— «Искупление Шоушенком»)\xa0— американский драматический фильм 1994 года режиссёра и сценариста Фрэнка Дарабонта, основанный на повести Стивена Кинга «Рита Хейуорт и спасение из Шоушенка». Он рассказывает историю банкира Энди Дюфрейна (Тим Роббинс), который приговорён к пожизненному заключению в государственной тюрьме Шоушенка за убийство своей жены и её любовника, несмотря на его заявления о невиновности. В течение следующих двух десятилетий он дружит с другим заключённым, контрабандистом Эллисом «Редом» Реддингом (Морган Фриман), и играет важную роль в операции по отмыванию денег, возглавляемой начальником тюрьмы Сэмюэлем Нортоном (Боб Гантон). Уильям Сэдлер, Клэнси Браун, Гил Беллоуз и Джеймс Уитмор появляются во второстепенных ролях.'}
{'Место': 

## Создание датасета


In [26]:
df = pd.DataFrame(movies_data)

df.set_index('Место', inplace=True)

## Вывод результатов


In [27]:
df.head(250)

,Название,Год,Режиссёр,Рейтинг,Описание
Место,,,,,
1,Побег из Шоушенка,1994,Фрэнк Дарабонт,"эпический фильм, историческая драма, тюремная ...",«Побе́г из Шоуше́нка» (англ. The Shawshank Red...
2,Крёстный отец,1972,Фрэнсис Форд Коппола,"эпический фильм, гангстерский фильм, фильм-тра...",Крёстный оте́ц может означать:
3,Тёмный рыцарь,2008,Кристофер Нолан,"эпический боевик, эпический фильм, супергеройс...",«Тёмный ры́царь» (англ. The Dark Knight) — аме...
4,Крёстный отец 2,1974,Фрэнсис Форд Коппола,"эпический фильм, гангстерский фильм, фильм-тра...",«Крёстный оте́ц 2» (англ. The Godfather Part I...
5,12 разгневанных мужчин,1957,Сидни Люмет,"юридическая драма, психологическая драма, дете...",«12 разгневанных мужчин» (англ. Twelve Angry M...
...,...,...,...,...,...
246,Прислуга,2011,Тейт Тейлор,"историческая драма, драма","Дома́шняя прислу́га (слу́ги, дво́рня, дворо́вы..."
247,Форма голоса,2016,Наоко Ямада,"аниме, фильм про взросление, психологическая д...",«Форма голоса» (яп. 聲の形 Коэ но катати) — манга...
248,Сука любовь,2000,Алехандро Гонсалес Иньярриту,"фильм-трагедия, драма, триллер",Сука любовь:


In [28]:
df.to_csv('imdb_rating')